In [64]:
import sys
import numpy as np
import artm
print artm.version()

from os import path, mkdir
from datetime import datetime
%matplotlib inline
sys.path.insert(0, '..\\modules\\helpers')
from plot_helper import PlotMaker
from config_helper import ConfigPaths
from print_helper import PrintHelper
from sklearn.decomposition import PCA

0.8.1


In [2]:
config = ConfigPaths('config.cfg')
plot_maker = PlotMaker()
printer = PrintHelper()

In [3]:
print config.models_file_name

Q:\\topic_modeling\\csi_science_collections.git\experiments\UCI_filtered_ngramm_trimmed_without_names\25_11_decor\models.txt


In [4]:
models_file = open(config.models_file_name, 'a')

In [5]:
def create_model(current_dictionary, n_topics, n_doc_passes, seed_value, n_top_tokens, p_mass_threshold):    
    print '[{}] creating model'.format(datetime.now())
    model = artm.ARTM(num_topics=n_topics, dictionary=current_dictionary, cache_theta=True, seed=seed_value, 
                  class_ids={'ngramm': 1.0, 'author_id': 0.0, 'author': 0.0, 
                             'post_tag': 0.0, 'projects': 0.0, 'category': 0.0,
                             'following_users': 0.0})
    model.num_document_passes = n_doc_passes
    add_scores_to_model(model, n_top_tokens=n_top_tokens, p_mass_threshold=p_mass_threshold)
    return model


def add_scores_to_model(artm_model, n_top_tokens, p_mass_threshold):
    print '[{}] adding scores'.format(datetime.now())
    artm_model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                      use_unigram_document_model=False,
                                      dictionary=dictionary))
    artm_model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score', class_id='ngramm'))
    artm_model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    artm_model.scores.add(artm.TopicKernelScore(name='topic_kernel_score', class_id='ngramm', 
                                                probability_mass_threshold=p_mass_threshold))
    artm_model.scores.add(artm.TopTokensScore(name='top_tokens_score', class_id='ngramm', num_tokens=n_top_tokens))

In [6]:
def process_one_model(dictionary, _n_topics, _n_doc_passes, _seed_value, _n_top_tokens, _p_mass_threshold, _n_iterations,
                     _model_name=''):
    print '[{}] processing model'.format(datetime.now())
    model = create_model(current_dictionary=dictionary, n_topics=_n_topics, n_doc_passes=_n_doc_passes, seed_value=_seed_value,
                         n_top_tokens=_n_top_tokens, p_mass_threshold=_p_mass_threshold)
    model = fit_one_model(model, _n_iterations, _model_name)
    return model
    
def fit_one_model(model, _n_iterations, _model_name=''): 
    print '[{}] fitting'.format(datetime.now())
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=_n_iterations)
    print '[{}] outputting'.format(datetime.now())
    printer.print_artm_model(model, _model_name, _n_iterations, output_file=models_file)
    model_pics_file_name =  path.join(config.experiment_path, _model_name)
    plot_maker.make_tm_plots(model, model_pics_file_name)
    model_output_file_name = path.join(config.experiment_path, _model_name + '.txt')
    printer.print_scores(model, _model_name, _n_iterations, model_output_file_name)
    printer.print_top_tokens(model, model_output_file_name)
    return model

In [7]:
def save_model(_model, _model_name): 
    print '[{}] saving model'.format(datetime.now())
    model_output_file_name = path.join(config.models_archive_path, _model_name)
    _model.save(filename=model_output_file_name+'_saved_p_wt', model_name=_model_name+'p_wt')
    _model.save(filename=model_output_file_name+'_saved_n_wt', model_name=_model_name+'n_wt')

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path=config.dataset_path,
                                        data_format='bow_uci',
                                        collection_name=config.collection_name,
                                        target_folder=config.output_batches_path)
dictionary = artm.Dictionary()
dictionary.gather(data_path=config.output_batches_path,
                  vocab_file_path=config.vocabulary_path)
dictionary.save(dictionary_path=config.dictionary_path)
dictionary.save_text(dictionary_path=config.dictionary_path + '.txt')
dictionary.load_text(dictionary_path=config.dictionary_path + '.txt')

In [8]:
batch_vectorizer = artm.BatchVectorizer(data_path=config.output_batches_path,
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path=config.dictionary_path + '.dict')

In [ ]:
# dictionary.filter(min_tf=5, max_tf=2000, min_df_rate=0.01, max_df_rate=0.9)

In [ ]:
# простая модель без регуляризаторов 
# темы вроде нормаьльные, возможно в некоторых темах странные слова-выбросы (18)

In [ ]:
model_no_reg2000_5 = create_model(current_dictionary=dictionary, n_topics=2000, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25)
model_no_reg2000_5 = fit_one_model(model_no_reg2000_5, _n_iterations=25, _model_name='model_no_reg2000_5')

In [ ]:
# модель c sparse регуляризаторами 

In [9]:
tmp_model = create_model(current_dictionary=dictionary, n_topics=2000, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25)
tmp_model = fit_one_model(tmp_model, _n_iterations=25, _model_name='model_sparse_reg_1')
tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_regularizer', class_ids=['ngramm']))
tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_regularizer'))
tmp_model.regularizers['sparse_phi_regularizer'].tau = -0.5
tmp_model.regularizers['sparse_theta_regularizer'].tau = -0.5
tmp_model = fit_one_model(tmp_model, _n_iterations=20, _model_name='model_sparse_reg_1')
model_sparse_reg_1 = tmp_model; tmp_model = None

[2016-11-25 15:07:08.806000] creating model
[2016-11-25 15:07:15.104000] adding scores
[2016-11-25 15:07:15.309000] fitting
[2016-11-25 15:22:56.268000] outputting
name = model_sparse_reg_1, n_topics = 2000, n_doc_passes = 5, seed_value = 100, n_iterations = 25, n_top_tokens = 15, p_threshold = 0.25

[2016-11-25 15:24:11.561000] fitting
[2016-11-25 15:37:05.276000] outputting
name = model_sparse_reg_1, n_topics = 2000, n_doc_passes = 5, seed_value = 100, n_iterations = 20, n_top_tokens = 15, p_threshold = 0.25
sparse_theta_regularizer, tau = -0.5
sparse_phi_regularizer, tau = -0.5



In [ ]:
tmp_model = create_model(current_dictionary=dictionary, n_topics=2000, n_doc_passes=5, seed_value=100,
                         n_top_tokens=15, p_mass_threshold=0.25)
tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_regularizer'))
tmp_model.regularizers['sparse_theta_regularizer'].tau = -0.5
tmp_model = fit_one_model(tmp_model, _n_iterations=15, _model_name='model_decor_sparse_t_reg_2')
model_decor_sparse_t_reg_2 = tmp_model; tmp_model = None

In [10]:
from scipy.spatial import ConvexHull
phi = model_sparse_reg_1.get_phi()
print(phi)

                      topic_0  topic_1   topic_2   topic_3   topic_4  \
книги                     0.0      0.0  0.000000  0.000000  0.000000   
fuchs                     0.0      0.0  0.000000  0.000000  0.000000   
preobrazhensky            0.0      0.0  0.000000  0.000000  0.000000   
tabachnikov               0.0      0.0  0.000000  0.000000  0.000000   
автограф                  0.0      0.0  0.000000  0.000000  0.000000   
автор                     0.0      0.0  0.000000  0.000000  0.000000   
кейс                      0.0      0.0  0.000000  0.000000  0.000000   
математика                0.0      0.0  0.000000  0.000000  0.000000   
образование               0.0      0.0  0.000785  0.000000  0.000000   
красота                   0.0      0.0  0.000000  0.000000  0.000000   
книга                     0.0      0.0  0.000000  0.000000  0.000000   
университетский_курс      0.0      0.0  0.000000  0.000000  0.000000   
дорога                    0.0      0.0  0.000000  0.000000  0.00

In [ ]:
def get_distance(topic, other_topic):
    # take elements that are both nonzero
    nonzero_indices = np.intersect1d(np.where(topic != 0), np.where(other_topic != 0))
    topic_cut = topic[nonzero_indices]
    other_topic_cut = other_topic[nonzero_indices]
    dist = 0.5 * (np.log(topic_cut) - np.log(other_topic_cut)) * (topic_cut - other_topic_cut)
    return dist

In [ ]:
def take_closest_topic(_phi, topic_idx):
    topic = _phi[_phi.colphi.columnspic_idx]]
    distances = {}
    for col_idx in _phi.columns:
        current_topic = _phi[_phi.columns[col_idx]]
        distance = get_distance(topic, current_topic)
        distances[col_idx] = distance
    return distances

In [49]:
a = np.where(phi[phi.columns[1]] != 0)
b = np.where(phi[phi.columns[2]] != 0)
c = np.intersect1d(a, b)
# print np.log(a)
# print np.where(np.divide(phi[phi.columns[1]],phi[phi.columns[1]]) == 1)
print c, a[0].shape
print phi[phi.columns[1]][c]

[  63  107  177  188  223  370  678  953 1175 1283 1837 2948 3572 4504] (186L,)
часть            0.013075
половина         0.001130
имя              0.001657
пример           0.013693
период           0.005560
старший          0.003414
использование    0.000970
история          0.001588
специалист       0.003345
точность         0.003387
устройство       0.007328
основание        0.000626
критерий         0.003288
союз             0.007442
Name: topic_1, dtype: float32


In [ ]:
phi_t = phi.transpose()

In [51]:
phi_cut = phi[phi.columns[0 : 10]]
print phi_cut

                      topic_0  topic_1   topic_2   topic_3   topic_4  \
книги                     0.0      0.0  0.000000  0.000000  0.000000   
fuchs                     0.0      0.0  0.000000  0.000000  0.000000   
preobrazhensky            0.0      0.0  0.000000  0.000000  0.000000   
tabachnikov               0.0      0.0  0.000000  0.000000  0.000000   
автограф                  0.0      0.0  0.000000  0.000000  0.000000   
автор                     0.0      0.0  0.000000  0.000000  0.000000   
кейс                      0.0      0.0  0.000000  0.000000  0.000000   
математика                0.0      0.0  0.000000  0.000000  0.000000   
образование               0.0      0.0  0.000785  0.000000  0.000000   
красота                   0.0      0.0  0.000000  0.000000  0.000000   
книга                     0.0      0.0  0.000000  0.000000  0.000000   
университетский_курс      0.0      0.0  0.000000  0.000000  0.000000   
дорога                    0.0      0.0  0.000000  0.000000  0.00

In [ ]:
print(np.sum(phi[phi.columns[0]]))
print(phi_t.shape)

In [58]:
print(phi_cut.transpose)

<bound method DataFrame.transpose of                       topic_0  topic_1   topic_2   topic_3   topic_4  \
книги                     0.0      0.0  0.000000  0.000000  0.000000   
fuchs                     0.0      0.0  0.000000  0.000000  0.000000   
preobrazhensky            0.0      0.0  0.000000  0.000000  0.000000   
tabachnikov               0.0      0.0  0.000000  0.000000  0.000000   
автограф                  0.0      0.0  0.000000  0.000000  0.000000   
автор                     0.0      0.0  0.000000  0.000000  0.000000   
кейс                      0.0      0.0  0.000000  0.000000  0.000000   
математика                0.0      0.0  0.000000  0.000000  0.000000   
образование               0.0      0.0  0.000785  0.000000  0.000000   
красота                   0.0      0.0  0.000000  0.000000  0.000000   
книга                     0.0      0.0  0.000000  0.000000  0.000000   
университетский_курс      0.0      0.0  0.000000  0.000000  0.000000   
дорога                    0

In [63]:
points = phi.transpose()

In [ ]:
print '[{}] PCA fitting started '.format(datetime.now())
model = PCA(n_components=200).fit(points)
print '[{}] PCA fitting finished '.format(datetime.now())
proj_vertices = model.transform(points)
print '[{}] PCA transforming finished '.format(datetime.now())
hull_kinda = ConvexHull(proj_vertices)
print '[{}] ConvexHull finished '.format(datetime.now())

[2016-11-25 19:40:41.905000] PCA fitting started 
[2016-11-25 19:42:02.329000] PCA fitting finished 
[2016-11-25 19:42:04.844000] PCA transforming finished 


In [62]:
hull = ConvexHull(phi_cut.transpose())
print hull.vertices

QhullError: QH6214 qhull input error: not enough points(10) to construct initial simplex (need 19322)

While executing:  | qhull i Qx Qt
Options selected for Qhull 2015.2.r 2016/01/18:
  run-id 1404989462  incidence  Qxact-merge  Qtriangulate  _zero-centrum
  Q3-no-merge-vertices-dim-high


In [ ]:
hull = ConvexHull(phi)
simplices = hull.simplices
vertices = hull.vertices

In [ ]:
hull[simplices, :]

In [ ]:
models_file.close()